This notebook is based on https://medium.com/analytics-vidhya/building-a-simple-chatbot-in-python-using-nltk-7c8c8215ac6e


https://www.geeksforgeeks.org/removing-stop-words-nltk-python/

y
http://josearcosaneas.github.io/python/r/procesamiento/lenguaje/2017/01/02/procesamiento-lenguaje-natural-0.html

In [0]:
import nltk
import numpy as np
import random
import string #used to process python strings

Importing the corpus to colab

In [0]:
from google.colab import files
uploaded = files.upload()

Saving santander_es.txt to santander_es.txt


Reading the data


In [0]:
f=open('santander_es.txt','r',errors = 'ignore')

raw=f.read()
raw=raw.lower() # converts to lowercase
nltk.download('punkt') # first-time use only
nltk.download('wordnet') # first-time use only
sent_tokens = nltk.sent_tokenize(raw)# converts to list of sentences 
word_tokens = nltk.word_tokenize(raw)# converts to list of words

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Example of sent_tokens and word_tokens

In [0]:
sent_tokens[:2]

['francisco de paula santander\nir a la navegación\nir a la búsqueda\nfrancisco de paula santander\nsantander by acevedo bernal.jpg\ncoat of arms of new granada.svg\n2.º presidente de la república de nueva granada\n10 de marzo de 1832-1 de abril de 1837\npredecesor\tjosé maría obando\nsucesor\tjosé ignacio de márquez\ncoat of arms of gran colombia (1821).svg\npresidente de la gran colombia\n(interino)\n7 de octubre de 1819-20 de febrero de 1827\npredecesor\tsimón bolívar\nsucesor\tsimón bolívar\nescudo ejercito nacional de colombia.svg\ngeneral del ejército nacional de colombia\n1817-1819\npredecesor\tsimón bolivar\nsucesor\tsimón bolívar\ninformación personal\nnacimiento\t2 de abril de 1792\nbandera de españa villa del rosario, virreinato de nueva granada, imperio español\nfallecimiento\t6 de mayo de 1840 (48 años)\nbandera de colombia bogotá, república de nueva granada\ncausa de la muerte\tcoledocolitiasis ver y modificar los datos en wikidata\nlugar de sepultura\tcementerio central 

In [0]:
word_tokens[:2]


['francisco', 'de']

Pre-processing the raw text

In [0]:
import nltk

#nltk.download()
from nltk import wordpunct_tokenize
from nltk.corpus import stopwords
import nltk
#from nltk.stem.porter import PorterStemmer
#from nltk.stem.Snowball import SnowballStemmer

stopw = [w.encode('utf-8') for w in stopwords.words('spanish')]
punc_lit = [u'.', u'[', ']', u',', u';', u'', u')', u'),', u' ', u'(']
stopw.extend(punc_lit)

In [0]:
from nltk.stem import SnowballStemmer

#print (SnowballStemmer.languages)
 
stemmer_spanish = SnowballStemmer('spanish')


def LemTokens(tokens):
    return [stemmer_spanish.stem(token) for token in tokens if token not in stopw]

remove_punct_dict = dict((ord(punct), None) for punct in string.punctuation)

def LemNormalize(text):
    return LemTokens(nltk.word_tokenize(text.lower().translate(remove_punct_dict)))

#Keyword matching

Next, we shall define a function for a greeting by the bot i.e if a user’s input is a greeting, the bot shall return a greeting response.ELIZA uses a simple keyword matching for greetings. We will utilize the same concept here.

In [0]:
GREETING_INPUTS = ("hola", "que más!", "cordial saludo", "hey!")

GREETING_RESPONSES = ["hola", "hey", "hola que hace!", "buenas las tenga", "a la orden?", "Es un gusto, pregunte pues!"]

def greeting(sentence):
 
    for word in sentence.split():
        if word.lower() in GREETING_INPUTS:
            return random.choice(GREETING_RESPONSES)

In [0]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [0]:
def response(user_response):
    robo_response=''
    sent_tokens.append(user_response)    
    TfidfVec = TfidfVectorizer(tokenizer=LemNormalize) #, stop_words='english')
    tfidf = TfidfVec.fit_transform(sent_tokens)
    vals = cosine_similarity(tfidf[-1], tfidf)
    idx=vals.argsort()[0][-2]
    flat = vals.flatten()
    flat.sort()
    req_tfidf = flat[-2]    
    if(req_tfidf==0):
        robo_response=robo_response+"Que pena no le entiendo nada... por favor pregunte otra cosa"
        return robo_response
    else:
        robo_response = robo_response+sent_tokens[idx]
        return robo_response

In [0]:
flag=True
print("ROBO: Soy un chatbot. Espero resolver todas sus preguntas, para salir escriba chao")
while(flag==True):
    user_response = input("You:")
    user_response=user_response.lower()
    if(user_response!='chao'):
        if(user_response=='gracias' or user_response=='Gracias!' ):
            flag=False
            print("ROBO: De nada..")
        else:
            if(greeting(user_response)!=None):
                print("ROBO: "+greeting(user_response))
            else:
                print("ROBO: ",end="")
                print(response(user_response))
                sent_tokens.remove(user_response)
    else:
        flag=False
        print("ROBO: Nos vemos!")

ROBO: Soy un chatbot. Espero resolver todas sus preguntas, para salir escriba chao
You:Cuando nació santander
ROBO: el departamento en el que nació se llama norte de santander, y al sur de éste está el departamento de santander.
You:que hizo Santander
ROBO: santander pidió a bolívar que arbitrara la situación en caracas, y el libertador le dio la razón a páez, lo que no fue bien recibido por los santanderistas.13​ por otro lado, después de ser conocida en bogotá la sublevación del coronel josé bustamante en lima, los enemigos del libertador organizaron festejos públicos en los que participó santander.
You:cuando fue presidente
ROBO: rechazando la reelección en la presidencia, cuando terminaba su mandato logro unir a sus seguidores en torno de un solo candidato y el triunfo fue para josé ignacio de márquez.
You:en que año fue presidente
ROBO: en diciembre de ese mismo año poco antes de morir bolívar escribió: “el no habernos arreglado con santander nos ha perjudicado a todos”.13​
presid